In [1]:
import sys
import bqplot
import pandas as pd
import ipywidgets as widgets
from restart import RestartModel
from restart import util
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from bqplot import pyplot as plt
import restart

gr_restart = RestartModel(
    config='wa_groups',
    population='oes',
    state='Washington',
    county='King',
    subpop='wa_groupings'
)

gr_model = gr_restart.model
gr_model.inventory.set_average_orders_per_period(gr_model.demand.demand_by_popsum1_total_rp1n_tc)

gr_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def gr_dashboard(backstop):
    gr_set_stock(backstop)
    
def gr_display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['EMTs', 'Firefighters', 'Other Public Safety']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def gr_set_stock(backstop):
    backstop = [backstop]
    gr_model.inventory.order(gr_model.inventory.inv_by_popsum1_total_rp1n_tc)
    gr_model.inventory.set_min_in_periods(backstop)
    gr_display_stock(gr_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(gr_model.demand.demand_per_unit_map_dn_um.df))
gr_pop = format_population(to_sheet(gr_model.population.population_pP_tr.df))
    
gr_out = widgets.interactive_output(gr_dashboard, {'backstop': gr_slider})
gr_sum_sheet = format_population(to_sheet(gr_model.population.pop_to_popsum1_per_unit_map_pp1_us.df))

In [2]:
import numpy as np
import ipysheet
import ipywidgets as widgets
from ipywidgets import Layout

restart = RestartModel(config='des', population='oes', state='Washington', county='King', subpop='wa_tier2_opt2')
model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = util.generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)

pop = format_population(to_sheet(model.population.population_pP_tr.df))

burn_chart = util.generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)
out = widgets.interactive_output(dashboard, {'backstop': slider})

wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))

essential_arr = restart.model.population.pop_to_popsum1_per_unit_map_pp1_us.array
population_labels = list(restart.model.population.pop_to_popsum1_per_unit_map_pp1_us.df.index)


essential_list = []

for row in essential_arr:
    if row[0] == 1:
        essential_list.append('Y')
    else:
        essential_list.append('N')

essential_df = pd.DataFrame(essential_list)
essential_df.insert(loc=0, column='Population', value=population_labels)
essential_df.columns = ['Population', 'Essential Worker']
essential_sheet = to_sheet(essential_df)
essential_sheet.row_headers = False

In [3]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['PPE Analysis']),
    v.Tab(children=['Methodology']),
    v.Tab(children=['Burn Rates']),
    v.Tab(children=['First Responders/Public Safety']),
    v.Tab(children=['Healthcare - Selected']),
                
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["PPE Stockpile"]),
                v.CardText(children=[
                    "First responders/public safety officers - manually selected SOC codes:"
                ]),
                gr_slider, gr_out,
                v.CardText(children=[
                    "Healthcare providers (inpatient and outpatient) - broad view (desired populations may be a percentage of this):"
                ]),
                slider, out,
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[(
                    """Group 1: Backstop to Large Providers""")]),
                v.CardText(children=[(
                    """These are the large hospitals and other facilities that take care of our 
                     sickest patients. With the fall, we will face the difficult challenge of  
                     both the flu season and the recurrence of COVID-19 as the climate worsens 
                     and people move indoors. This group will provide most of their own PPE,  
                     disinfection and other resources with the state acting as a backstop in extreme 
                     emergencies. Given the exponential nature of infection, we need this backstop  
                     because even the best predictions have a large variance. For example, if the 
                     disease doubles every week, then even a two week error in estimate will increase 
                     PPE requirements in COVID wards by 4x.""")]),           
                v.CardText(children=[(
                    """Group 2: Aggregate Demand of Smaller Providers""")]),       
                v.CardText(children=[(
                    """This second group consists of several different populations: 
                     Smaller health care providers such as hospitals with 299 beds or less, long-term care 
                     providers and nursing homes, behavioral health facilities, dentists, morticians, 
                     Federally Qualified Health Centers (FQHC), public health organizations, tribal clinics, 
                     independent physician practices, first responders including EMOs, police, and fire""")]),            
                v.CardText(children=[(
                    """We use two different methods to estimate these populations. More detailed surveys and  
                     census methods are also possible, so consider these methods as ways to get a broad measure 
                     of the scope of the problem.""")]),            
                v.CardText(children=[(
                    """Employee Classification (SOC) Analysis: """)]),                  
                v.CardText(children=[(
                    """SOC codes starting with "29-", "31-", and "33-" refer to, respectively, healthcare occupations, 
                     healthcare support occupations, and protection services. We estimate of the percentage of 
                     healthcare workers or healthcare-support workers fall into the Group 2 category, and simply 
                     scale all the numbers by that amount. It seems like a safe assumption that all the protection 
                     services would fall into this group.""")]),                  
                 v.CardText(children=[(
                    """This analysis provides the stockpile that you would need to 100% cover the group for 30 days.
                     A key policy decision is the size of the back stop needed. If you want to cover 50% of the 
                     demand for 30 days, then the figures would be half that.""")]),                     
            ]),
        ])
    ]),
                
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand"]),
                wa_burn_sheet, burn_chart
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["First Responders/Public Safety"]),
                v.CardSubtitle(children=["Population Breakdown"]),
                gr_pop,
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Healthcare - Broad"]),
                v.CardSubtitle(children=["Population Breakdown"]),
                pop,
            ])
        ])
    ]),
])

Tabs(children=[Tab(children=['PPE Analysis']), Tab(children=['Methodology']), Tab(children=['Burn Rates']), Ta…